In [ ]:
import os
import glob
import shutil
import itertools
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:

TRAIN_DATA_PATH = "/content/train"
TEST_DATA_PATH = "/content/test"

normalizer = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        normalizer
    ]),
    
    'test': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        normalizer
    ])
}

train_data = datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=data_transforms['train'])

test_data = datasets.ImageFolder(root=TEST_DATA_PATH, transform=data_transforms['test'])

In [ ]:
batch_size = 20

data_size = len(train_data)
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                         sampler=val_sampler)

In [ ]:
next(iter(train_loader))[1]

In [ ]:
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

In [ ]:
import tqdm

num_epochs = 5

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in tqdm.trange(1, num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
          
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    Returns: accuracy as a float value between 0 and 1
    """

    model.eval() # Evaluation mode
    
    for _, (x, y) in enumerate(loader):
      x, y = x.to(device), y.to(device)
      prediction = model(x)
      loss_val = loss(prediction, y)
      
      accuracy = (torch.argmax(prediction, dim=1) == y).sum().item() / prediction.shape[0]
    
    return accuracy

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = nn.Sequential(nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
                            nn.BatchNorm2d(6),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size = 2, stride = 2),
                            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                            nn.BatchNorm2d(16),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size = 2, stride = 2),
                            Flattener(),
                            nn.Linear(53824, 300),
                            nn.BatchNorm1d(300),
                            nn.ReLU(),
                            nn.Linear(300, 120),
                            nn.BatchNorm1d(120),
                            nn.ReLU(),
                            nn.Linear(120, 84),
                            nn.BatchNorm1d(84),
                            nn.ReLU(),
                            nn.Linear(84, 2),
                            nn.LogSoftmax(dim=-1))



model.type(torch.cuda.FloatTensor)
model.to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0.0001)

In [ ]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.0001)

loss_history, train_history, val_history = train_model(model, train_loader, val_loader, loss, optimizer, num_epochs)

 25%|██▌       | 1/4 [00:47<02:23, 47.95s/it]

Average loss: 0.371857, Train accuracy: 0.829900, Val accuracy: 0.888889


 50%|█████     | 2/4 [01:35<01:34, 47.45s/it]

Average loss: 0.278778, Train accuracy: 0.880263, Val accuracy: 0.777778


 75%|███████▌  | 3/4 [02:24<00:48, 48.43s/it]

Average loss: 0.228468, Train accuracy: 0.905727, Val accuracy: 0.888889


100%|██████████| 4/4 [03:11<00:00, 47.76s/it]

Average loss: 0.194483, Train accuracy: 0.922250, Val accuracy: 1.000000


In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=20)
final_test_accuracy = compute_accuracy(model, test_loader)
print("Final test accuracy -", final_test_accuracy)

Final test accuracy - 1.0


In [ ]:
torch.save({
        'epoch': num_epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss}, '/content/goto_model.model.pt')
    